In [103]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib as plt
from itertools import chain
import ast

In [104]:
df = pd.read_csv("data/formatted_data.csv")
df.shape

(672159, 30)

In [105]:
act_to_sat_conversion_math = {
    "36": 800, "35": 780, "34": 760, '33': 740, "32": 720, "31": 710,
    "30": 700, "29": 680, "28": 660, "27": 640, '26': 610, "25": 590,
    "24": 580, "23": 560, "22": 540, '21': 530, "20": 520, '19': 510,
    "18": 500, "17": 470, '16': 430, "15": 400, "14": 330, "13": 330,
    "12": 310, "11": 280, "10": 260,
}

act_to_sat_conversion_eng = {
    "72": 790, '71': 770, '70': 750, '69': 740, '68': 730,
    '67': 720, '66': 710, '65': 700, '64': 700, '63': 690,
    "62": 680, '61': 680, '60': 670, '59': 660, '58': 660,
    '57': 650, '56': 640, '55': 640, '54': 630, '53': 630,
    '52': 620, '51': 610, '50': 610, '49': 600, '48': 590,
    "47": 580, '46': 580, '45': 570, '44': 560, '43': 550,
    "42": 540, '41': 540, '40': 530, '39': 520, '38': 510,
    '37': 500, '36': 500, '35': 490, '34': 480, '33': 470,
    '32': 460, '31': 450, '30': 440, '29': 430, '28': 420,
    "27": 410, '26': 400, '25': 390, '24': 380, '23': 370,
    '22': 360, '21': 350, '20': 340, '19': 330, '18': 320,
    '17': 310, '16': 300, '15': 290, '14': 280
}

# Convert ACTM to SATM and keep the highest SATM
def convert_actm_to_satm(row):
    actm = row['ACTM']
    satm = row['SATM']
    
    if actm != '-' and actm in act_to_sat_conversion_math:
        converted_satm = act_to_sat_conversion_math[actm]
        if satm == '-' or converted_satm > int(satm):
            return converted_satm
    return int(satm) if satm != '-' else '-'

# Sum ACTR and ACTE, compare with SAT-ERW, and keep the highest
def convert_actr_acte_to_saterw(row):
    actr, acte = row['ACTR'], row['ACTE']
    sat_erw = row['SAT-ERW']
    if pd.isna(sat_erw):
        sat_erw = '-'

    if actr != '-' and acte != '-' and actr in act_to_sat_conversion_eng and acte in act_to_sat_conversion_eng:
        converted_sum = act_to_sat_conversion_eng[str(int(actr)+int(acte))]
        if sat_erw == '-' or converted_sum > int(sat_erw):
            return converted_sum
    return int(sat_erw) if sat_erw != '-' else '-'

df['SATM'] = df.apply(convert_actm_to_satm, axis=1)
df['SAT-ERW'] = df.apply(convert_actr_acte_to_saterw, axis=1)
df['Classes'] = df['Classes'].apply(ast.literal_eval)
df['Semester Grades'] = df['Semester Grades'].apply(ast.literal_eval)
df['Semester Points'] = df['Semester Points'].apply(ast.literal_eval)
df['CRN'] = df['CRN'].apply(ast.literal_eval)

In [106]:
Pidms_with_CHM2210 = df[df['Classes'].apply(lambda x: 'CHM2210' in x)]['Pidm'].unique()
df = df[df['Pidm'].isin(Pidms_with_CHM2210)]
df.head(4)

,Pidm,Semester,Admit_Level,Admit_College,Lastest_Major,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,HS GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Final GPA,Semester GPA,Semester Grades,Semester Points,Classes,CRN
140,10229,201505,UG,SA,Spanish,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),NaN,-,-,-,-,-,-,-,-,3.42,3.66,"[B+, A]","[3.33, 4.0]","[CHM2046, PSY2012]","[50015.0, 53874.0]"
141,10229,201508,UG,SA,Spanish,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),NaN,-,-,-,-,-,-,-,-,3.42,3.58,"[B, A, B+, A, W]","[3.0, 4.0, 3.33, 4.0, 0.0]","[PHY2053, PHY2053L, SYG2000, CHM2046L, CHM2210]","[80599.0, 80616.0, 84684.0, 88932.0, 92437.0]"
142,10229,201605,UG,SA,Spanish,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),NaN,-,-,-,-,-,-,-,-,3.42,2.84,"[C, A-]","[2.0, 3.67]","[HSC4537, ANT2000]","[52235.0, 53811.0]"
326,23613,202005,UG,SA,Biomedical Sciences,W,White,NaN,W White,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),4.23,-,-,-,-,-,-,-,-,3.79,NaN,"[WC, WC]","[0.0, 0.0]","[HSC4504, PHY2053]","[50797.0, 53263.0]"


In [107]:
def find_chm2210_semester(student_df):
    chm2210_row = student_df[student_df['Classes'].apply(lambda x: 'CHM2210' in x)]
    if not chm2210_row.empty:
        return chm2210_row['Semester'].min()
    return None


chm2210_semester_series = df.groupby('Pidm').apply(lambda x: find_chm2210_semester(x)).rename('CHM2210_Semester')

df = df.merge(chm2210_semester_series, on='Pidm')

filtered_df = df[df['Semester'] <= df['CHM2210_Semester']]

In [109]:
def get_chm2210_grades(classes, points, grades):
    if 'CHM2210' in classes:
        index = classes.index('CHM2210')
        return points[index], grades[index]
    return None, None

def find_chm2210_grades(student_df):
    for _, row in student_df.iterrows():
        points, grade = get_chm2210_grades(row['Classes'], row['Semester Points'], row['Semester Grades'])
        if points is not None and grade is not None:
            return points, grade
    return None, None

chm2210_grades = df.groupby('Pidm').apply(lambda x: find_chm2210_grades(x)).apply(pd.Series)
chm2210_grades.columns = ['CHM2210_Points', 'CHM2210_Grade']

final_df = df.merge(chm2210_grades, on='Pidm')

In [110]:
display(df[df['Pidm'] ==  10229])
final_df[final_df['Pidm'] ==  10229]
groupped_df = final_df.groupby('Pidm').agg({
    "Admit_Level": 'first',
    "Admit_College": 'first',
    "Lastest_Major": 'first',
    "Trump_Race": 'first',
    "Trump_Race_Desc": 'first',
    "MULTI": 'first',
    "Race": 'first',
    "NEW_ETHNICITY": 'first',
    "GENDER_Code": 'first',
    "GENDER": 'first',
    "HS GPA": 'first',
    "CITZ_CODE": 'first',
    "CITZ_DESC": 'first',
    
    "ACTE": 'first',
    "ACTR": 'first',
    "ACTS": 'first',
    "ACTM": 'first',
    "EACT": 'first',
    "SAT-ERW": 'first',
    "SATM": 'first',
    "SAT_TOTAL": 'first',
    'Final GPA': 'first',
    'Semester GPA': lambda x: list(x),
    'Semester Grades': lambda x: sum(x, []),
    'Semester Points': lambda x: sum(x, []),
    'Classes': lambda x: sum(x, []),
    'CRN': lambda x: sum(x, []),
    'CHM2210_Grade': 'first',
    'CHM2210_Points': 'first',
}).reset_index()

,Pidm,Semester,Admit_Level,Admit_College,Lastest_Major,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,CITZ_IND,CITZ_CODE,CITZ_DESC,HS GPA,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Final GPA,Semester GPA,Semester Grades,Semester Points,Classes,CRN,CHM2210_Semester
0,10229,201505,UG,SA,Spanish,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),NaN,-,-,-,-,-,-,-,-,3.42,3.66,"[B+, A]","[3.33, 4.0]","[CHM2046, PSY2012]","[50015.0, 53874.0]",201508
1,10229,201508,UG,SA,Spanish,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),NaN,-,-,-,-,-,-,-,-,3.42,3.58,"[B, A, B+, A, W]","[3.0, 4.0, 3.33, 4.0, 0.0]","[PHY2053, PHY2053L, SYG2000, CHM2046L, CHM2210]","[80599.0, 80616.0, 84684.0, 88932.0, 92437.0]",201508
2,10229,201605,UG,SA,Spanish,B,Black or African American,NaN,B Black or African American,Not Hispanic or Latino,F,Female,Y,C,Citizen (USA),NaN,-,-,-,-,-,-,-,-,3.42,2.84,"[C, A-]","[2.0, 3.67]","[HSC4537, ANT2000]","[52235.0, 53811.0]",201508


,Pidm,Admit_Level,Admit_College,Lastest_Major,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,HS GPA,CITZ_CODE,CITZ_DESC,ACTE,ACTR,ACTS,ACTM,EACT,SAT-ERW,SATM,SAT_TOTAL,Final GPA,Semester GPA,Semester Grades,Semester Points,Classes,CRN,CHM2210_Grade,CHM2210_Points
0,10229,UG,SA,Spanish,B,Black or African American,None,B Black or African American,Not Hispanic or Latino,F,Female,NaN,C,Citizen (USA),-,-,-,-,-,-,-,-,3.42,"[3.66, 3.58, 2.84]","[B+, A, B, A, B+, A, W, C, A-]","[3.33, 4.0, 3.0, 4.0, 3.33, 4.0, 0.0, 2.0, 3.67]","[CHM2046, PSY2012, PHY2053, PHY2053L, SYG2000,...","[50015.0, 53874.0, 80599.0, 80616.0, 84684.0, ...",W,0.00
1,23613,UG,SA,Biomedical Sciences,W,White,None,W White,Not Hispanic or Latino,F,Female,4.23,C,Citizen (USA),-,-,-,-,-,-,-,-,3.79,"[nan, 3.16, 4.0, 3.67, 3.89, 4.0, 4.0, 3.67]","[WC, WC, B+, B, A, A+, A, A+, A, B, A, A+, A-,...","[0.0, 0.0, 3.33, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0,...","[HSC4504, PHY2053, PHY2053, HSC4504, BSC2085, ...","[50797.0, 53263.0, 80441.0, 83033.0, 82366.0, ...",A,4.00
2,63387,UG,SA,Health Sciences,H,Hispanic or Latino,None,W White,Hispanic or Latino,F,Female,NaN,C,Citizen (USA),-,-,-,-,-,-,-,-,2.32,"[4.0, nan, 2.0, nan, 2.11, 2.67, 2.44, nan, 3....","[A, CF, CF, C, D, B, CF, CF, CF, C-, B-, C, A-...","[4.0, 0.0, 0.0, 2.0, 1.0, 3.0, 0.0, 0.0, 0.0, ...","[ENC2210, CLT3040, LAE4414, BSC2010, HSC4537, ...","[21755.0, 57634.0, 59840.0, 82844.0, 86012.0, ...",W,0.00
3,134328,UG,SA,Cell and Molecular Biology,W,White,None,W White,Not Hispanic or Latino,M,Male,3.00,C,Citizen (USA),32,29,32,27,30,680,640,-,1.95,"[2.5, 4.0, 3.34, 1.16, 4.0, 0.33]","[C, B, A, B, A-, D, D+, F, C+, A, S, D, F, F]","[2.0, 3.0, 4.0, 3.0, 3.67, 1.0, 1.33, 0.0, 2.3...","[BSC2010L, BSC2010, ENC1101, PSY2012, ARC2211,...","[11143.0, 23005.0, 11691.0, 52696.0, 52892.0, ...",D,1.00
4,360849,UG,SP,English,W,White,None,W White,Not Hispanic or Latino,M,Male,NaN,C,Citizen (USA),-,-,-,-,-,-,-,-,2.09,"[3.0, 2.58, 1.67, 3.11, 2.87, 3.67, 0.92, 0.75]","[W, B, D+, B, B, B, W, D+, F, A-, C+, B+, A-, ...","[0.0, 3.0, 1.33, 3.0, 3.0, 3.0, 0.0, 1.33, 0.0...","[CHM2210, PHY2053, BSC2010, BSC2010L, PHY2054,...","[53832.0, 56079.0, 87410.0, 87412.0, 90672.0, ...",W,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15074,3877042,UG,AC,Psychology,B,Black or African American,None,B Black or African American,Not Hispanic or Latino,F,Female,4.08,C,Citizen (USA),25,27,20,25,24,620,590,-,3.38,"[3.75, 3.2, 3.27]","[B, A+, A, A, B, W, A+, A, A, D, B, B+, B, A+, B]","[3.0, 4.0, 4.0, 4.0, 3.0, 0.0, 4.0, 4.0, 4.0, ...","[HSC4537, CHM2046, BSC2086, BSC2086L, PCB3023L...","[82097.0, 82555.0, 83986.0, 83987.0, 10655.0, ...",B+,3.33
15075,3878121,UG,AC,Biology,W,White,None,W White,Not Hispanic or Latino,F,Female,3.65,C,Citizen (USA),-,-,-,-,-,680,690,1370,3.81,"[4.0, 3.94, 3.33]","[A, A, A+, A, A, A+, A, A-, A, B, B, A]","[4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.67, 4.0,...","[HSC4537, HSC4631, ENC1102, STA2023, HSC4573, ...","[82097.0, 82314.0, 88675.0, 11560.0, 13118.0, ...",A-,3.67
15076,3878148,UG,AC,Biology,W,White,None,W White,Not Hispanic or Latino,M,Male,NaN,C,Citizen (USA),-,-,-,-,-,610,600,1210,2.40,[2.4],"[C, C, C, A, C]","[2.0, 2.0, 2.0, 4.0, 2.0]","[HSC4630, BSC2011, BSC2011L, CCJ3024, CHM2210]","[82310.0, 84158.0, 84165.0, 84562.0, 84858.0]",C,2.00
15077,3879752,UG,AC,Marine Biology,W,White,None,W White,Not Hispanic or Latino,F,Female,3.92,C,Citizen (USA),31,34,26,24,29,700,580,-,2.81,"[2.78, 2.83]","[B+, B-, C+, C, C+, B+, A-]","[3.33, 2.67, 2.33, 2.0, 2.33, 3.33, 3.67]","[PCB3063, BSC3312, MAC2241, EVR3218, CHM2210, ...","[81419.0, 85547.0, 86259.0, 10692.0, 10865.0, ...",C+,2.33


In [111]:
groupped_df.head(5)

,Pidm,Admit_Level,Admit_College,Lastest_Major,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,GENDER_Code,GENDER,HS GPA,CITZ_CODE,CITZ_DESC,ACTE,ACTR,ACTS,ACTM,EACT,SAT-ERW,SATM,SAT_TOTAL,Final GPA,Semester GPA,Semester Grades,Semester Points,Classes,CRN,CHM2210_Grade,CHM2210_Points
0,10229,UG,SA,Spanish,B,Black or African American,None,B Black or African American,Not Hispanic or Latino,F,Female,NaN,C,Citizen (USA),-,-,-,-,-,-,-,-,3.42,"[3.66, 3.58, 2.84]","[B+, A, B, A, B+, A, W, C, A-]","[3.33, 4.0, 3.0, 4.0, 3.33, 4.0, 0.0, 2.0, 3.67]","[CHM2046, PSY2012, PHY2053, PHY2053L, SYG2000,...","[50015.0, 53874.0, 80599.0, 80616.0, 84684.0, ...",W,0.0
1,23613,UG,SA,Biomedical Sciences,W,White,None,W White,Not Hispanic or Latino,F,Female,4.23,C,Citizen (USA),-,-,-,-,-,-,-,-,3.79,"[nan, 3.16, 4.0, 3.67, 3.89, 4.0, 4.0, 3.67]","[WC, WC, B+, B, A, A+, A, A+, A, B, A, A+, A-,...","[0.0, 0.0, 3.33, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0,...","[HSC4504, PHY2053, PHY2053, HSC4504, BSC2085, ...","[50797.0, 53263.0, 80441.0, 83033.0, 82366.0, ...",A,4.0
2,63387,UG,SA,Health Sciences,H,Hispanic or Latino,None,W White,Hispanic or Latino,F,Female,NaN,C,Citizen (USA),-,-,-,-,-,-,-,-,2.32,"[4.0, nan, 2.0, nan, 2.11, 2.67, 2.44, nan, 3....","[A, CF, CF, C, D, B, CF, CF, CF, C-, B-, C, A-...","[4.0, 0.0, 0.0, 2.0, 1.0, 3.0, 0.0, 0.0, 0.0, ...","[ENC2210, CLT3040, LAE4414, BSC2010, HSC4537, ...","[21755.0, 57634.0, 59840.0, 82844.0, 86012.0, ...",W,0.0
3,134328,UG,SA,Cell and Molecular Biology,W,White,None,W White,Not Hispanic or Latino,M,Male,3.00,C,Citizen (USA),32,29,32,27,30,680,640,-,1.95,"[2.5, 4.0, 3.34, 1.16, 4.0, 0.33]","[C, B, A, B, A-, D, D+, F, C+, A, S, D, F, F]","[2.0, 3.0, 4.0, 3.0, 3.67, 1.0, 1.33, 0.0, 2.3...","[BSC2010L, BSC2010, ENC1101, PSY2012, ARC2211,...","[11143.0, 23005.0, 11691.0, 52696.0, 52892.0, ...",D,1.0
4,360849,UG,SP,English,W,White,None,W White,Not Hispanic or Latino,M,Male,NaN,C,Citizen (USA),-,-,-,-,-,-,-,-,2.09,"[3.0, 2.58, 1.67, 3.11, 2.87, 3.67, 0.92, 0.75]","[W, B, D+, B, B, B, W, D+, F, A-, C+, B+, A-, ...","[0.0, 3.0, 1.33, 3.0, 3.0, 3.0, 0.0, 1.33, 0.0...","[CHM2210, PHY2053, BSC2010, BSC2010L, PHY2054,...","[53832.0, 56079.0, 87410.0, 87412.0, 90672.0, ...",W,0.0


In [112]:
all_classes = sorted(set(chain.from_iterable(groupped_df['Classes'])))

def create_one_hot(classes, all_classes):
    one_hot_vector = [0] * len(all_classes)
    for class_name in classes:
        if class_name in all_classes:
            one_hot_vector[all_classes.index(class_name)] = 1
    return one_hot_vector

groupped_df['One_Hot_Classes'] = groupped_df['Classes'].apply(lambda x: create_one_hot(x, all_classes))


one_hot_df = pd.DataFrame(groupped_df['One_Hot_Classes'].tolist(), columns=all_classes, index=groupped_df['Pidm'])

In [113]:
print(one_hot_df.shape)
one_hot_df.head(3)

(15079, 3616)


ACG2021  ACG2071  ACG3074  ACG3103  ACG3113  ACG3341  ACG3401  ACG4123  \
Pidm                                                                            
10229        0        0        0        0        0        0        0        0   
23613        0        0        0        0        0        0        0        0   
63387        0        0        0        0        0        0        0        0   

       ACG4351  ACG4632  ACG4642  ACG4684  ACG4931  ACG4940  ACG5205  ACG5505  \
Pidm                                                                            
10229        0        0        0        0        0        0        0        0   
23613        0        0        0        0        0        0        0        0   
63387        0        0        0        0        0        0        0        0   

       ACG5675  ACG6026  ACG6457  ACG6636  ACG6688  ACG6875  ACG6936  ADE6385  \
Pidm                                                                            
10229        0        0        0        0        0        0        0        0   
23613        0        0        0        0        0        0        0        0   
63387        0        0        0        0        0        0        0        0   

       ADV2214  ADV3008  ADV3101  ADV3300  ADV3500  ADV4301  ADV4310  ADV4600  \
Pidm                                                                            
10229        0        0        0        0        0        0        0        0   
23613        0        0        0        0        0        0        0        0   
63387        0        0        0        0        0        0        0        0   

       ADV4800  ADV4940  AFA2000  AFA2380  AFA3110  AFA4150  AFA4335  AFA4430  \
Pidm                                                                            
10229        0        0        0        0        0        0        0        0   
23613        0        0        0        0        0        0        0        0   
63387        0        0        0        0        0        0        0        0   

       AFA4713  AFA4900  AFA4931  AFH3100  AFH3200  AFH6300  AFR1101  AFR1120  \
Pidm                                                                            
10229        0        0        0        0        0        0        0        0   
23613        0        0        0        0        0        0        0        0   
63387        0        0        0        0        0        0        0        0   

       AFR2001  AFR2130  AFR2140  AFR3220  AFR3231  AFR4201  AFR4211  AFS2250  \
Pidm                                                                            
10229        0        0        0        0        0        0        0        0   
23613        0        0        0        0        0        0        0        0   
63387        0        0        0        0        0        0        0        0   

       AFS3251  AMH2010  AMH2020  AMH3110  AMH3130  AMH3140  AMH3160  AMH3170  \
Pidm                                                                            
10229        0        0        0        0        0        0        0        0   
23613        0        0        0        0        0        0        0        0   
63387        0        0        0        0        0        0        0        0   

       AMH3231  AMH3270  AMH3341  AMH3390  AMH3402  AMH3403  AMH3421  AMH3423  \
Pidm                                                                            
10229        0        0        0        0        0        0        0        0   
23613        0        0        0        0        0        0        0        0   
63387        0        0        0        0        0        0        0        0   

       AMH3512  AMH3530  AMH3533  AMH3545  AMH3562  AMH3571  AMH3572  AML3031  \
Pidm                                                                            
10229        0        0        0        0        0        0        0        0   
23613        0        0        0        0        0        0        0        0   
63387        0        0      